<a href="https://colab.research.google.com/github/Kimminsu-ds/ADP/blob/main/%ED%86%B5%EA%B3%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제32회 ADP 실기 대비 - 통계분석 정리



In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 표시 설정

import warnings
warnings.filterwarnings(action='ignore')

## 28회 기출문제



### 4.
- status: 생존여부(death: 죽음, event lost: 생존)
- company: 회사구분

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p4_.csv")
df.head(3)

,time(month),status,company
0,1,event lost,X
1,2,event lost,X
2,3,event lost,X


### 5.
- 고객들의 시식 전 구매의사와 시식 후 구매의사에 대한 응답을 나타낸 데이터

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p5_.csv")
df.head(3)

,data,userId
0,시식전_유__시식후_유,user_1
1,시식전_유__시식후_유,user_2
2,시식전_유__시식후_유,user_3


In [ ]:
df['시식 전'] = df['data'].apply(lambda x: x.split("__")[0][-1])
df['시식 후'] = df['data'].apply(lambda x: x.split("__")[1][-1])
df.head(3)

,data,userId,시식 전,시식 후
0,시식전_유__시식후_유,user_1,유,유
1,시식전_유__시식후_유,user_2,유,유
2,시식전_유__시식후_유,user_3,유,유


#### 5-1.
- 시식 여부가 구매의사에 영향을 주는지를 검정하기 위한 가설을 설정하시오.

In [ ]:
# 귀무가설(H0): 시식 전/후의 구매의사는 차이가 없다.
# 대립가설(H1): 시식 전/후의 구매의사는 차이가 있다.

#### 5-2.
- 가설을 검정하고 결과를 분석하시오.

In [ ]:
## 맥니머 검정(McNemar's test)
# 맥니머 검정은 이항변수인 두 변수의 대응관계(Paired)가 있는 데이터 분포의 차이를 검정할 때 사용한다.
# [Tip] 2 x 2 범주형 변수의 시점 차이를 검증해야 할 때 사용

from statsmodels.stats.contingency_tables import mcnemar

table = pd.crosstab(df['시식 전'], df['시식 후'])
print(table)

# exact값이 True이면 이항분포, False이면 카이제곱분포를 사용
mc = mcnemar(table.values, exact=False, correction=False)

print(f"검정통계량: {round(mc.statistic, 3)}, 유의확률(p-value): {round(mc.pvalue, 3)}")

시식 후   무   유
시식 전        
무     12  17
유      8  23
검정통계량: 3.24, 유의확률(p-value): 0.072


### 7.
- 데이터: 연령, 콜레스테롤, 몸무게

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p7.csv")
df.head(3)

,age,Cholesterol,weight
0,65,69.0,111.0
1,54,117.0,81.0
2,61,86.2,72.0


#### 7-1.
- 몸무게를 제어한다고 생각하고, 연령과 콜레스테롤 간 상관계수 및 유의확률을 구하여라.

In [ ]:
## 편상관관계(Partial correlations)
# 제3의 변수를 통제한 상태에서 관심을 가지는 두 변수의 상관관계를 분석하는 것
# !pip install pingouin

import pingouin

result = pingouin.partial_corr(data=df, x='age', y='Cholesterol', covar='weight')
result

,n,r,CI95%,p-val
pearson,333,-0.134317,"[-0.24, -0.03]",0.014316


In [ ]:
print("몸무게를 제어하는 경우")
print(f" 연령과 콜레스테롤 간 상관계수는 {round(result['r'].values[0], 3)}, 유의확률(p-value)는 {round(result['p-val'].values[0], 3)}")

몸무게를 제어하는 경우
 연령과 콜레스테롤 간 상관계수는 -0.134, 유의확률(p-value)는 0.014


#### 7-2.
- 상관계수를 유의수준 0.05 하에서 검정하라.

In [ ]:
# 귀무가설(H0): 두 변수 간의 편상관계수는 0이다.
# 대립가설(H1): 두 변수 간의 편상관계수는 0이 아니다. (즉, 편상관관계가 있다.)

# 유의수준 0.05하에서 유의확률(p-value)이 유의수준보다 낮으므로, 귀무가설을 기각한다.
# 즉, 연령과 콜레스테롤 변수 간 편상관관계가 있다.

## 29회 기출문제

### 5.
- 제품 A의 불량률은 3%이다.
- 25개의 제품을 뽑았을 때 3개의 제품이 불량품일 확률을 구하시오.
- (소수점 다섯째 자리에서 반올림)

In [ ]:
## 이항분포
from scipy.stats import binom

n = 25 # 시행횟수
k = 3 # 성공횟수
p = 0.03 # 성공할 확률

# 이항분포의 기댓값, 분산 구하기 - binom.stats(n, p)
E, V = binom.stats(n, p)

# 이항분포의 확률질량함수 구하기 - binom.pmf(x, n, p)
pmf = binom.pmf(k, n, p)

# 이항분포의 누적확률질량함수 구하기 - binom.cdf(x, n, p)
cdf = binom.cdf(k, n, p)

print(f"기댓값: {round(E, 4)}, 분산: {round(V, 4)}, 확률질량함수: {round(pmf, 4)}, 누적확률질량함수: {round(cdf, 4)}")

기댓값: 0.75, 분산: 0.7275, 확률질량함수: 0.0318, 누적확률질량함수: 0.9938


### 6.
- C사 생산 제품 1,000개 중 양품이 600개, D사 생산 제품 500개 중 양품이 200개 이다.
- 두 회사의 양품률에 차이가 있는지 검정하여라.

In [ ]:
## 카이제곱분포 - 동질성 검정 (Test of Homogeneity)

# 데이터 준비
table = pd.DataFrame({"회사" : ["C", "D"], "정상": [600, 200], "불량" : [400, 300]}).set_index("회사")

# [가설설정]
# 귀무가설(H0): 두 회사의 양품률에는 차이가 없다. (동일하다)
# 대립가설(H1): 두 회사의 양품률에는 차이가 있다. (동일하지 않다)

from scipy.stats import chi2_contingency
chi2, p, df, expec = chi2_contingency(table, correction=False)

# 유의수준 0.05하에서 동질성 검정 결과, 유의확률이 0.05보다 작으므로 귀무가설을 기각하고 대립가설을 채택한다.
# 즉, C사와 D사의 양품률에는 차이가 있다.
print(f"검정통계량: {round(chi2, 4)}, p-value: {round(p, 4)}")

검정통계량: 53.5714, p-value: 0.0


### 8.
- 한 공장의 3개의 생산라인별 생산률과 불량률은 아래 표와 같다.

|구분|L1|L2|L3|
|-|-|-|-
|생산률|13%|37%|50%|
|불량률|1.1%|2.1%|3.3%|

- 불량품이 나왔을 때, L1 생산라인에서 생산되었을 확률을 구하시오.
- (소수점 둘째자리에서 반올림)

In [ ]:
## 베이즈 정리
# P(A) = 불량품이 나올 확률 = P(L1) + P(L2) + P(L3)
 # P(L1): L1 생산라인에서 불량품이 생산될 확률
 # P(L2): L2 생산라인에서 불량품이 생산될 확률
 # P(L3): L3 생산라인에서 불량품이 생산될 확률

P_L1 = 0.13 * 0.011
P_L2 = 0.37 * 0.021
P_L3 = 0.50 * 0.033
P_A = P_L1 + P_L2 + P_L3

# P(L1|A) = 불량품이 나왔을 때, L1 생산라인에서 생산되었을 확률 = P(L1) / P(A)
P_L1_A = P_L1 / P_A

print(f"불량품이 나왔을 때, L1 생산라인에서 생산되었을 확률: {round(P_L1_A, 1)}")

불량품이 나왔을 때, L1 생산라인에서 생산되었을 확률: 0.1


## 30회 기출문제

### 5.
- 3개의 공장에서 생산한 제품을 1, 2, 3 지역으로 가능한 조건에 맞게 최대로 배송한다.
- 각 공장과 각 지역 사이의 1개 제품 배송 비용이 아래와 같을 때, 최소 배송 비용은 얼마인가?

|공장|지역1|지역2|지역3|총 생산량|
|-|-|-|-|-|
|공장A|20만원|5만원|35만원|75|
|공장B|15만원|2만원|20만원|55|
|공장C|3만원|14만원|17만원|60|
|총 수요량|80|50|60|-|

In [ ]:
## 선형계획법

### 6.
- 데이터: 3개 연령층(청년/장년/노년)의 헤드셋에 대한 선호도(1/2/3) 조사

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/30/p6.csv")
df.head(3)

,나이,선호도
0,청년,2
1,장년,2
2,청년,2


#### 6-1.
- 연령대별로 헤드셋 선호도에 차이가 있는지를 검정하기 위한 귀무가설과 연구가설을 설정

In [ ]:
# 귀무가설(H0): 연령대별로 헤드셋에 대한 선호도 차이가 없다.
# 대립가설(H1): 연령대별로 헤드셋에 대한 선호도 차이가 있다.

#### 6-2.
- 유의수준 0.05 하에서 가설검정을 수행

In [ ]:
table = pd.crosstab(df['나이'], df['선호도'])
table

선호도,1,2,3
나이,,,
노년,35,32,25
장년,25,23,17
청년,47,44,29


In [ ]:
## 카이제곱분포 - 독립성 검정 (Test of independence)
from scipy.stats import chi2_contingency
chi2, p, df, expec = chi2_contingency(table)

# 유의수준 0.05하에서 독립성 검정 결과, 유의확률이 0.05보다 크므로 귀무가설을 채택한다.
# 즉, 연령대별로 헤드셋에 대한 선호도 차이가 없다.
print(f"검정통계량: {round(chi2, 4)}, p-value: {round(p, 4)}")

검정통계량: 0.2651, p-value: 0.992


### 7.
- 아이를 6명 가진 가족이 5가족이고, 아들/딸을 출생할 확률은 각각 0.5이다.

#### 7-1.
- 딸이 4명 이상인 가족이 3가족 이상일 확률을 구하여라.

In [ ]:
## 이항분포
from scipy.stats import binom

# 딸이 4명 이상일 확률: (1) 딸4명 확률 + (2) 딸5명 확률 + (3) 딸6명 확률
n = 6 # 자녀 수
p = 0.5 # 딸을 출생할 확률

p_4 = binom.pmf(n=6, k=4, p=0.5) # (1) 딸4명 확률
p_5 = binom.pmf(n=6, k=5, p=0.5) # (2) 딸5명 확률
p_6 = binom.pmf(n=6, k=6, p=0.5) # (3) 딸6명 확률

# 이항분포의 확률질량함수
D_4 = p_4 + p_5 + p_6
D_4 # 딸이 4명 이상일 확률

0.34375

In [ ]:
# 딸이 4명 이상인 가족이 3가족 이상일 확률 : (1) 딸4명 3가족 + (2) 딸4명 4가족 + (3) 딸4명 5가족

n = 5 # 가족 수
p = D_4 # 딸이 4명 이상일 확률

# 이항분포의 누적질량함수
p_3 = binom.pmf(n=5, k=3, p=p) # (1) 딸4명 3가족
p_4 = binom.pmf(n=5, k=4, p=p) # (2) 딸4명 4가족
p_5 = binom.pmf(n=5, k=5, p=p) # (3) 딸4명 5가족

F_3 = p_3 + p_4 + p_5
F_3

0.2255459427833557

#### 7-2.
- 5가족 중 딸이 4명 이상인 가족이 몇 가족으로 기대할 수 있는가?

In [ ]:
# D_4: 딸이 4명 이상일 확률

n = 5 # 가족 수

# 이항분포의 기댓값, 분산
E, V = binom.stats(n, p)
print(f"이항분포의 기댓값: {round(E,3)}, 분산: {round(V, 3)}")

이항분포의 기댓값: 1.719, 분산: 1.128


## 31회 기출문제

### 5. 평균이 $\theta$이고 분산이 100인 정규분포의 사전분포가 100, 256일 때 120의 값을 가지는 데이터가 있을 경우 사후 평균을 구하시오.

In [ ]:
# theta를 샘플링된 제품의 무게인 120으로 가정하며, n=1로 두고 사후분포를 계산한다.

# 사전확률분포 모수
prior_mean = 100
prior_var = 256

mu = 120 # theta를 120으로 가정
var = 100 # 사후분포의 분산

var / (prior_var + var) * prior_mean + (prior_var) / (prior_var + var) * mu

114.38202247191012

### 7.
- 영업사원 A, B, C, D, E의 계약 성사유무(0:미계약, 1:계약)를 나타낸 데이터이다.
- 영업사원들의 평균 계약 성사 건수가 일치한지 통계 검정을 수행하시오.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_7.csv", index_col='Unnamed: 0')
df.head(3)

,A,B,C,D,E
contract_1,1,0,1,1,1
contract_2,0,1,0,1,0
contract_3,1,0,0,0,0


In [ ]:
## 코크란(Cochran's Q) 검정
# 귀무가설(H0): 영업사원들의 평균 계약 성사 건수는 차이가 없다.
# 대립가설(H1): 영업사원들의 평균 계약 성사 건수는 차이가 있다.

from statsmodels.stats.contingency_tables import cochrans_q
ccq = cochrans_q(df)

# 유의수준 0.05 하에서 유의확률(p-value)이 0.05 보다 크므로 귀무가설을 채택한다.
# 즉, 영업사원들의 평균 계약 성사 건수는 차이가 없다.
print(f"검정통계량: {round(ccq.statistic, 3)}, 유의확률(p-value): {round(ccq.pvalue, 3)}")

검정통계량: 4.516, 유의확률(p-value): 0.341


### 8.
- 유기견이 하루에 평균적으로 2.2마리 발생한다.

#### 8-1.
- 유기견이 하루에 한 마리도 버려지지 않을 확률을 구하시오.

In [ ]:
## 포아송 분포
from scipy.stats import poisson

# 단위시간당 평균 사건 발생 횟수
lmbda = 2.2

# 포아송분포의 확률질량함수
# 사건이 한 번도 밣생하지 않을 확률
prob_0 = poisson.pmf(0, lmbda)
print(f"유기견이 하루에 한 마리도 버려지지 않을 확률: {round(prob_0, 3)}")

유기견이 하루에 한 마리도 버려지지 않을 확률: 0.111


#### 8-2.
- 유기견이 하루에 적어도 2마리 이상 버려질 확률을 구하시오.

In [ ]:
# 포아송분포의 누적확률질량함수
prob_least_2 = 1 - poisson.cdf(1, lmbda)
print(f"유기견이 하루에 적어도 2마리 이상 버려질 확률: {round(prob_least_2, 3)}")

유기견이 하루에 적어도 2마리 이상 버려질 확률: 0.645
